In [1]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [ ]:
%mkdir ch21
!touch ch21/__init__.py

## 20.0 序言

- 类元编程是指在运行时创建或定制类的技艺
- 

## 21.1 类工厂函数

###### 示例 21-2 record_factory.py:一个简单的类工厂函数

In [ ]:
%%writefile ch21/record_factory.py
def record_factory(cls_name, field_names):
  try:
    field_names = field_names.replace(',', ' ').split()  # ❶ 这里体现了鸭子类型:尝试在逗号或空格处拆分 field_names;如果失败,那么假定 field_names 本就是可迭代的对象, 一个元素对应一个属性名
  except AttributeError: # 不能调用.replace或.split方法
    pass # 假定field_names本就是标识符组成的序列
  field_names = tuple(field_names)  # ➋ 使用属性名构建元组,这将成为新建类的 __slots__ 属性; 此外, 这么做还设定了拆包和字符串表示形式中各字段的顺序

  def __init__(self, *args, **kwargs):  # ➌ 新建类的 __init__ 方法
    attrs = dict(zip(self.__slots__, args))
    attrs.update(kwargs)
    for name, value in attrs.items():
      setattr(self, name, value)

  def __iter__(self):  # ➍ 实现 __iter__ 函数,把类的实例变成可迭代的对象;按照 __slots__ 设定的顺序产出字段值
    for name in self.__slots__: 
      yield getattr(self, name)

  def __repr__(self):  # ➎ 迭代 __slots__ 和 self, 生成友好的字符串表示形式
    values = ', '.join('{}={!r}'.format(*i) for i
    in zip(self.__slots__, self))
    return '{}({})'.format(self.__class__.__name__, values)
  
  cls_attrs = dict( __slots__ = field_names,  # ➏ 组建类属性字典
                    __init__ = __init__,
                    __iter__ = __iter__,
                    __repr__ = __repr__)
  
  return type(cls_name, (object,), cls_attrs)  # ➐ 调用 type 构造方法,构建新类,然后将其返回

Writing ch21/record_factory.py


###### 示例 21-1 测试 record_factory 函数,一个简单的类工厂函数

In [ ]:
from ch21.record_factory import record_factory

In [ ]:
Dog = record_factory('Dog', 'name weight owner')  # ❶ 这个工厂函数的签名与 namedtuple 类似
rex = Dog('Rex', 30, 'Bob')
rex  # ❷ 友好的字符串表示形式

Dog(name='Rex', weight=30, owner='Bob')

In [ ]:
name, weight, _ = rex  # ➌ 实例是可迭代的对象,因此赋值时可以便利地拆包
name, weight

('Rex', 30)

In [ ]:
"{2}'s dog weighs {1}kg".format(*rex)  # ❹ 传给 format 等函数时也可以拆包

"Bob's dog weighs 30kg"

In [ ]:
rex.weight = 32  # ➎ 记录实例是可变的对象
rex

Dog(name='Rex', weight=32, owner='Bob')

In [ ]:
Dog.__mro__  # ❻ 新建的类继承自 object, 与工厂函数没有关系

(ch21.record_factory.Dog, object)

## 21.2 定制描述符的类装饰器

###### 示例 21-4 model_v6.py: 一个类装饰器

In [ ]:
%%writefile ch21/model_v6.py
import abc


class AutoStorage:  # ➊ AutoStorage 类提供了之前 Quantity 描述符的大部分功能
  __counter = 0

  def __init__(self):
    cls = self.__class__
    prefix = cls.__name__
    index = cls.__counter
    self.storage_name = '_{}#{}'.format(prefix, index)
    cls.__counter += 1

  def __get__(self, instance, owner):
    if instance is None:
      return self
    else:
      return getattr(instance, self.storage_name)

  def __set__(self, instance, value):
      setattr(instance, self.storage_name, value)  # ➋ ......验证除外


class Validated(abc.ABC, AutoStorage):  # ➌ Validated 是抽象类,不过也继承自 AutoStorage 类
  def __set__(self, instance, value):
    value = self.validate(instance, value)  # ➍ __set__ 方法把验证操作委托给 validate 方法......
    super().__set__(instance, value)  # ➎ ......然后把返回的 value 传给超类的 __set__ 方法,存储值
    
  @abc.abstractmethod
  def validate(self, instance, value):  # ➏ validate 是类的抽象方法
    """return validated value or raise ValueError"""

  
class Quantity(Validated):  # ➐ Quantity 和 NonBlank 都继承自 Validated 类
  """a number greater than zero"""
  def validate(self, instance, value):
    if value <= 0:
      raise ValueError('value must be > 0')
    return value


class NonBlank(Validated):
  """a string with at least one non-space character"""
  def validate(self, instance, value):
    value = value.strip()
    if len(value) == 0:
      raise ValueError('value cannot be empty or blank')
    return value  # ➑ 要求具体的 validate 方法返回验证后的值,借机可以清理、转换或规范化接收的数据。这里把 value 首尾的空白去掉,然后将其返回


def entity(cls):  # 装饰器的参数是一个类
  for key, attr in cls.__dict__.items():  # 迭代存储类属性的字典
    if isinstance(attr, Validated):  # 如果属性是 Validated 描述符的实例
      type_name = type(attr).__name__
      attr.storage_name = '_{}#{}'.format(type_name, key)  # ......使用描述符类的名称和托管属性的名称命名 storage_name
  return cls  # ➎ 返回修改后的类

Writing ch21/model_v6.py


###### 示例 21-3 bulkfood_v6.py: 使用 `Quantity` 和 `NonBlank` 描述符的 `LineItem` 类

In [ ]:
%%writefile ch21/bulkfood_v6.py
import ch21.model_v6 as model

@model.entity  # ➊ 此类唯一的变化是添加了装饰器
class LineItem:
  description = model.NonBlank()
  weight = model.Quantity()
  price = model.Quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price
  
  def subtotal(self):
    return self.weight * self.price

Overwriting ch21/bulkfood_v6.py


###### 示例 21-5 bulkfood_v6.py: 描述符中新 `storage_name` 属性

In [ ]:
from ch21.bulkfood_v6 import LineItem
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [ ]:
LineItem.description.storage_name

'_NonBlank#description'

In [ ]:
raisins.description

'Golden raisins'

In [ ]:
getattr(raisins, '_NonBlank#description')

'Golden raisins'

## 21.3 导入时和运行时比较

### 理解计算时间的练习

###### 示例 21-6evaltime.py:按顺序写出输出的序号标记 <[N]>

In [ ]:
%%writefile ch21/evaltime.py
from ch21.evalsupport import deco_alpha

print('<[1]> evaltime module start')


class ClassOne():
  print('<[2]> ClassOne body')
  def __init__(self):
    print('<[3]> ClassOne.__init__')

  def __del__(self):
    print('<[4]> ClassOne.__del__')

  def method_x(self):
    print('<[5]> ClassOne.method_x')

  class ClassTwo(object):
    print('<[6]> ClassTwo body')


@deco_alpha
class ClassThree():
  print('<[7]> ClassThree body')
  def method_y(self):
    print('<[8]> ClassThree.method_y')


class ClassFour(ClassThree):
  print('<[9]> ClassFour body')
  def method_y(self):
    print('<[10]> ClassFour.method_y')

if __name__ == '__main__':
  print('<[11]> ClassOne tests', 30 * '.')
  one = ClassOne()
  one.method_x()
  print('<[12]> ClassThree tests', 30 * '.')
  three = ClassThree()
  three.method_y()
  print('<[13]> ClassFour tests', 30 * '.')
  four = ClassFour()
  four.method_y()
  
print('<[14]> evaltime module end')

Overwriting ch21/evaltime.py


###### 示例 21-7 evalsupport.py:evaltime.py 导入的模块

In [ ]:
%%writefile ch21/evalsupport.py
print('<[100]> evalsupport module start')

def deco_alpha(cls):
  print('<[200]> deco_alpha')

  def inner_1(self):
    print('<[300]> deco_alpha:inner_1')

  cls.method_y = inner_1
  return cls

class MetaAleph(type):
  print('<[400]> MetaAleph body')

  def __init__(cls, name, bases, dic):
    print('<[500]> MetaAleph.__init__')

    def inner_2(self):
      print('<[600]> MetaAleph.__init__:inner_2')
  
    cls.method_z = inner_2

print('<[700]> evalsupport module end')

Overwriting ch21/evalsupport.py


###### 示例 21-8 场景 1:在 Python 控制台中导入 `evaltime` 模块

In [ ]:
import ch21.evaltime

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body
<[14]> evaltime module end


```shell
<[100]> evalsupport module start  # ❶ evalsupport 模块中的所有顶层代码在导入模块时运行;解释器会编译 deco_alpha 函数,但是不会执行定义体
<[400]> MetaAleph body  # ❷ MetaAleph 类的定义体运行了
<[700]> evalsupport module end
<[1]> evaltime module start  
<[2]> ClassOne body  # ❸ 每个类的定义体都执行了......
<[6]> ClassTwo body  # ❹ ......包括嵌套的类
<[7]> ClassThree body  
<[200]> deco_alpha  # ❺ 先计算被装饰的类 ClassThree 的定义体,然后运行装饰器函数
<[9]> ClassFour body
<[14]> evaltime module end  # ❻ 在这个场景中,evaltime 模块是导入的,因此不会运行 if__name__ == '__main__': 块
```

###### 示例 21-9 场景 2:在 shell 中运行 evaltime.py

In [ ]:
!python3 -m ch21.evaltime

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body
<[11]> ClassOne tests ..............................
<[3]> ClassOne.__init__
<[5]> ClassOne.method_x
<[12]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1
<[13]> ClassFour tests ..............................
<[10]> ClassFour.method_y
<[14]> evaltime module end
<[4]> ClassOne.__del__


```shell
<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body  # ❶ 目前为止,输出与示例 21-8 相同
<[11]> ClassOne tests ..............................
<[3]> ClassOne.__init__  # ❷ 类的标准行为
<[5]> ClassOne.method_x
<[12]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1  # ❸ deco_alpha 装饰器修改了 ClassThree.method_y 方法,因此调用 three.method_y() 时会运行 inner_1 函数的定义体
<[13]> ClassFour tests ..............................
<[10]> ClassFour.method_y
<[14]> evaltime module end
<[4]> ClassOne.__del__  # ❹ 只有程序结束时,绑定在全局变量 one 上的 ClassOne 实例才会被垃圾回收程序回收
```

## 21.4 元类基础知识

- Python 中的类是 `type` 类的实例。 也就是说, `type` 是大多数内置的类和用户定义的类的元类

In [ ]:
'spam'.__class__

str

In [ ]:
str.__class__

type

In [ ]:
from ch21.bulkfood_v6 import LineItem
LineItem.__class__

type

In [ ]:
type.__class__

type

- 除了 `type`,标准库中还有一些别的元类,例如 `ABCMeta` 和 `Enum`

In [ ]:
import collections
collections.Iterable.__class__

abc.ABCMeta

In [ ]:
import abc
abc.ABCMeta.__class__

type

In [ ]:
abc.ABCMeta.__mro__

(abc.ABCMeta, type, object)

### 理解元类计算时间的练习

###### 示例 21-10 evaltime_meta.py: `ClassFive` 是 `MetaAleph` 元类的实例

In [ ]:
%%writefile ch21/evaltime_meta.py
from ch21.evalsupport import deco_alpha
from ch21.evalsupport import MetaAleph

print('<[1]> evaltime_meta module start')


@deco_alpha
class ClassThree():
  print('<[2]> ClassThree body')

  def method_y(self):
    print('<[3]> ClassThree.method_y')


class ClassFour(ClassThree):
  print('<[4]> ClassFour body')

  def method_y(self):
    print('<[5]> ClassFour.method_y')


class ClassFive(metaclass=MetaAleph):
  print('<[6]> ClassFive body')

  def __init__(self):
    print('<[7]> ClassFive.__init__')

  def method_z(self):
    print('<[8]> ClassFive.method_z')


class ClassSix(ClassFive):
  print('<[9]> ClassSix body')
  def method_z(self):
    print('<[10]> ClassSix.method_z')


if __name__ == '__main__':
  print('<[11]> ClassThree tests', 30 * '.')
  three = ClassThree()
  three.method_y()
  print('<[12]> ClassFour tests', 30 * '.')
  four = ClassFour()
  four.method_y()
  print('<[13]> ClassFive tests', 30 * '.')
  five = ClassFive()
  five.method_z()
  print('<[14]> ClassSix tests', 30 * '.')
  six = ClassSix()
  six.method_z()

print('<[15]> evaltime_meta module end')

Writing ch21/evaltime_meta.py


###### 示例 21-11: 场景 3:在 Python 控制台中导入 `evaltime_meta` 模块

In [ ]:
import ch21.evaltime_meta

<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[15]> evaltime_meta module end


```shell
<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__  # ➊ 与场景 1 的关键区别是,创建 ClassFive 时调用了MetaAleph.__init__ 方法
<[9]> ClassSix body
<[500]> MetaAleph.__init__  # 创建 ClassFive 的子类 ClassSix 时也调用了MetaAleph.__init__ 方法
<[15]> evaltime_meta module end
```

###### 示例 21-13 场景 4:在 shell 中运行 evaltime_meta.py

In [ ]:
!python3 -m ch21.evaltime_meta

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[11]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1
<[12]> ClassFour tests ..............................
<[5]> ClassFour.method_y
<[13]> ClassFive tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[14]> ClassSix tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[15]> evaltime_meta module end


```shell
<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[11]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1  # ❶ 装饰器依附到 ClassThree 类上之后,method_y 方法被替换成inner_1 方法......
<[12]> ClassFour tests ..............................
<[5]> ClassFour.method_y  # ❷ 虽然 ClassFour 是 ClassThree 的子类,但是没有依附装饰器的 ClassFour 类却不受影响
<[13]> ClassFive tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2  # ❸ MetaAleph 类的 __init__ 方法把 ClassFive.method_z 方法替换成 inner_2 函数
<[14]> ClassSix tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2  # ❹ ClassFive 的子类 ClassSix 也是一样,method_z 方法被替换成 inner_2 函数
<[15]> evaltime_meta module end
```

## 21.5 定制描述符的元类

###### 示例 21-14 `bulkfood_v7.py`:有元类的支持,继承 `model.Entity` 类即可


In [ ]:
%%writefile ch21/bulkfood_v7.py
import ch21.model_v7 as model


class LineItem(model.Entity):  # ➊ LineItem 是 model.Entity 的子类
  description = model.NonBlank()
  weight = model.Quantity()
  price = model.Quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price


Overwriting ch21/bulkfood_v7.py


###### 示例 21-15  model_v7.py: `EntityMeta` 元类以及它的一个实例


In [ ]:
%%writefile ch21/model_v7.py
import abc


class AutoStorage:  # ➊ AutoStorage 类提供了之前 Quantity 描述符的大部分功能
  __counter = 0

  def __init__(self):
    cls = self.__class__
    prefix = cls.__name__
    index = cls.__counter
    self.storage_name = '_{}#{}'.format(prefix, index)
    cls.__counter += 1

  def __get__(self, instance, owner):
    if instance is None:
      return self
    else:
      return getattr(instance, self.storage_name)

  def __set__(self, instance, value):
      setattr(instance, self.storage_name, value)  # ➋ ......验证除外


class Validated(abc.ABC, AutoStorage):  # ➌ Validated 是抽象类,不过也继承自 AutoStorage 类
  def __set__(self, instance, value):
    value = self.validate(instance, value)  # ➍ __set__ 方法把验证操作委托给 validate 方法......
    super().__set__(instance, value)  # ➎ ......然后把返回的 value 传给超类的 __set__ 方法,存储值
    
  @abc.abstractmethod
  def validate(self, instance, value):  # ➏ validate 是类的抽象方法
    """return validated value or raise ValueError"""


class Quantity(Validated):  # ➐ Quantity 和 NonBlank 都继承自 Validated 类
  """a number greater than zero"""
  def validate(self, instance, value):
    if value <= 0:
      raise ValueError('value must be > 0')
    return value


class NonBlank(Validated):
  """a string with at least one non-space character"""
  def validate(self, instance, value):
    value = value.strip()
    if len(value) == 0:
      raise ValueError('value cannot be empty or blank')
    return value  # ➑ 要求具体的 validate 方法返回验证后的值,借机可以清理、转换或规范化接收的数据。这里把 value 首尾的空白去掉,然后将其返回


class EntityMeta(type):
  """元类,用于创建带有验证字段的业务实体"""
  def __init__(cls, name, bases, attr_dict):
    super().__init__(name, bases, attr_dict)  # 在超类(在这里是 type)上调用 __init__ 方法
    for key, attr in attr_dict.items():   # 与示例 21-4 中 @entity 装饰器的逻辑一样
      if isinstance(attr, Validated):
        type_name = type(attr).__name__
        attr.storage_name = '_{}#{}'.format(type_name, key)

class Entity(metaclass=EntityMeta):  # 这个类的存在只是为了用起来便利:这个模块的用户直接继承Entity 类即可,无需关心 EntityMeta 元类,甚至不用知道它的存在
  """带有验证字段的业务实体"""


Overwriting ch21/model_v7.py


In [ ]:
from ch21.bulkfood_v7 import LineItem
raisins = LineItem('Golden raisins', 10, 6.95)

In [ ]:
LineItem.description.storage_name

'_NonBlank#description'

In [ ]:
raisins.description

'Golden raisins'

In [ ]:
getattr(raisins, '_NonBlank#description')

'Golden raisins'

## 21.6 元类的特殊方法 `__prepare__`

###### 示例 21-16 model_v8.py: 这一版 EntityMeta 元类用到了 `__prepare__` 方法,而且为 `Entity` 类定义了 `field_names` 类方法


In [ ]:
%%writefile ch21/model_v8.py
import collections
import abc


class AutoStorage:  # ➊ AutoStorage 类提供了之前 Quantity 描述符的大部分功能
  __counter = 0

  def __init__(self):
    cls = self.__class__
    prefix = cls.__name__
    index = cls.__counter
    self.storage_name = '_{}#{}'.format(prefix, index)
    cls.__counter += 1

  def __get__(self, instance, owner):
    if instance is None:
      return self
    else:
      return getattr(instance, self.storage_name)

  def __set__(self, instance, value):
      setattr(instance, self.storage_name, value)  # ➋ ......验证除外


class Validated(abc.ABC, AutoStorage):  # ➌ Validated 是抽象类,不过也继承自 AutoStorage 类
  def __set__(self, instance, value):
    value = self.validate(instance, value)  # ➍ __set__ 方法把验证操作委托给 validate 方法......
    super().__set__(instance, value)  # ➎ ......然后把返回的 value 传给超类的 __set__ 方法,存储值
    
  @abc.abstractmethod
  def validate(self, instance, value):  # ➏ validate 是类的抽象方法
    """return validated value or raise ValueError"""


class Quantity(Validated):  # ➐ Quantity 和 NonBlank 都继承自 Validated 类
  """a number greater than zero"""
  def validate(self, instance, value):
    if value <= 0:
      raise ValueError('value must be > 0')
    return value


class NonBlank(Validated):
  """a string with at least one non-space character"""
  def validate(self, instance, value):
    value = value.strip()
    if len(value) == 0:
      raise ValueError('value cannot be empty or blank')
    return value  # ➑ 要求具体的 validate 方法返回验证后的值,借机可以清理、转换或规范化接收的数据。这里把 value 首尾的空白去掉,然后将其返回

class EntityMeta(type):
  """元类,用于创建带有验证字段的业务实体"""

  @classmethod
  def __prepare__(cls, name, bases):
    return collections.OrderedDict()  # ➊ 返回一个空的 OrderedDict 实例,类属性将存储在里面

  def __init__(cls, name, bases, attr_dict):
    super().__init__(name, bases, attr_dict)
    cls._field_names = []  # ➋ 在要构建的类中创建一个 _field_names 属性
    for key, attr in attr_dict.items():  # ➌ 这里的 attr_dict 是那个OrderedDict 对象,由解释器在调用 __init__ 方法之前调用__prepare__ 方法时获得。因此,这个 for 循环会按照添加属性的顺序迭代属性
      if isinstance(attr, Validated):
        type_name = type(attr).__name__
        attr.storage_name = '_{}#{}'.format(type_name, key)
        cls._field_names.append(key)  # ➍ 把找到的各个 Validated 字段添加到 _field_names 属性中


class Entity(metaclass=EntityMeta):
  """带有验证字段的业务实体"""
  @classmethod
  def field_names(cls):  # ➎ field_names 类方法的作用简单:按照添加字段的顺序产出字段的名称
    for name in cls._field_names:
      yield name

Overwriting ch21/model_v8.py


###### 示例 21-17 bulkfood_v8.py: 无需修改 `LineItem` 类, `field_names` 方法继承自 `model.Entity` 类

In [ ]:
%%writefile ch21/bulkfood_v8.py
import ch21.model_v8 as model


class LineItem(model.Entity):  # ➊ LineItem 是 model.Entity 的子类
  description = model.NonBlank()
  weight = model.Quantity()
  price = model.Quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price

Overwriting ch21/bulkfood_v8.py


In [ ]:
from ch21.bulkfood_v8 import LineItem  # 类是对象，其导入时会进行初始化，会调用其元类的 __prepare__, __new__, __init__ 等方法
for name in LineItem.field_names():
  print(name)

description
weight
price


- 由类创建实例时，相当于调用类中的 `__call__` 方法